<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-experiments/haystack_conditional_router_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/deepset-ai/haystack.git@connection_router_v2#egg=farm-haystack[preview]

### 1) Let's create a custom component we'll use in our example

In [ ]:
from haystack.preview import component

@component
class Echo:

    component.output_types(output=str)
    def run(self, query: str):
        return {"output":query}

### 2) The ConditionalRouter with the routes defined

In [ ]:
from haystack.preview.components.routers import ConditionalRouter
routes = [
        {
            "condition": "{{query|length > 20}}",
            "output": "{{query}}",
            "output_name": "complex_query",
            "output_type": str,
        },
        {
            "condition": "{{query|length <= 20}}",
            "output": "{{query}}",
            "output_name": "simple_query",
            "output_type": str,
        },
    ]
router = ConditionalRouter(routes=routes)

### 3) Define the pipeline

In [ ]:
from haystack.preview.components.builders.prompt_builder import PromptBuilder
from haystack.preview import Pipeline

pipe = Pipeline()
pipe.add_component("router", router)
pipe.add_component("prompt_builder", PromptBuilder("Tell me more about {{query}}"))
pipe.add_component("echo", Echo())
pipe.connect("router.complex_query", "echo.query")
pipe.connect("router.simple_query", "prompt_builder.query")

### 4) And run it
#### Change query variable to a longer text (> 20 chars), see what happens

In [ ]:
query = "Berlin"
pipe.run(data={"router": {"query": query}})

{'prompt_builder': {'prompt': 'Tell me more about Berlin'}}

### 5) Serialize the pipeline

In [ ]:
fp = open("haystack-router-prompt.yaml", "w")
pipe.dump(fp)
fp.close()

### 6) Another ConditionalRouter with a mix of complex and simple str objects

In [ ]:
from typing import List
from haystack.preview.dataclasses import ChatMessage

routes_new = [
        {
            "condition": "{{messages[-1].content|length > 20}}",
            "output": "{{messages}}",
            "output_name": "complex_query",
            "output_type": List[ChatMessage],
        },
        {
            "condition": "{{messages[-1].content|length <= 20}}",
            "output": "{{messages[-1].content}}",
            "output_name": "simple_query",
            "output_type": str,
        },
    ]
new_router = ConditionalRouter(routes=routes_new)

### 7) EchoChatMessage used in our experiments

In [ ]:
@component
class EchoChatMessage:

    component.output_types(output=List[ChatMessage])
    def run(self, query: List[ChatMessage]):
        return {"output":query}

### 8) New pipeline to handle ChatMessage

In [ ]:
new_pipe = Pipeline()
new_pipe.add_component("router", new_router)
new_pipe.add_component("prompt_builder", PromptBuilder("Tell me more about {{query}}"))
new_pipe.add_component("echo", EchoChatMessage())
new_pipe.connect("router.complex_query", "echo.query")
new_pipe.connect("router.simple_query", "prompt_builder.query")

### 9) Run the new Pipeline, again change `usr_msg` and see what happens

In [ ]:
from haystack.preview.dataclasses import ChatMessage

user_msg = ChatMessage.from_user("Berlin") # change to longer text to trigger complex query path
messages = [ChatMessage.from_system("Speak in short A1 German sentences"), user_msg]

new_pipe.run(data={"router": {"messages": messages}})

{'prompt_builder': {'prompt': 'Tell me more about Berlin'}}

### 10) Serialize this Pipeline
#### Notice the difference between this pipeline serialized form and the previous

In [ ]:
fp_new = open("haystack-router-complex-prompt.yaml", "w")
new_pipe.dump(fp_new)
fp_new.close()

### 11) Load complex pipeline and run it again
#### Compare results to previous run, they should match

In [ ]:
complex_pipe_file = open("haystack-router-complex-prompt.yaml", "r")
new_pipe_restored = Pipeline.load(complex_pipe_file)
complex_pipe_file.close()


new_pipe_restored.run(data={"router": {"messages": messages}})

{'prompt_builder': {'prompt': 'Tell me more about Berlin'}}